In [157]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import seaborn as sns
import scipy
import os
import statsmodels.api as sm
import scipy.stats as stats
import statsmodels.formula.api as smf

if os.getcwd().split("\\")[-1] == "homework":
    os.chdir("../")
    
import cmds.custom_portfolio_management_helper as cpm
import cmds.portfolio_management_helper as pm

# Homework \#7 

## Case: Grantham, Mayo, and Van Otterloo, 2012: Estimating the Equity Risk Premium) [9-211-051].

# 1 GMO

This section is not graded, and you do not need to submit your answers. But you are expected to consider these issues and be ready to discuss them.



## 1.1. GMO's approach.
(a) Why does GMO believe they can more easily predict long-run than short-run asset class performance?
(b) What predicting variables does the case mention are used by GMO? Does this fit with the goal of long-run forecasts?
(c) How has this approach led to contrarian positions?
(d) How does this approach raise business risk and managerial career risk?



## 1.2. The market environment.
(a) We often estimate the market risk premium by looking at a large sample of historic data. What reasons does the case give to be skeptical that the market risk premium will be as high in the future as it has been over the past 50 years?
(b) In 2007, GMO forecasts real excess equity returns will be negative. What are the biggest drivers of their pessimistic conditional forecast relative to the unconditional forecast. (See Exhibit 9.)
(c) In the 2011 forecast, what components has GMO revised most relative to 2007? Now how does their conditional forecast compare to the unconditional? (See Exhibit 10.)



## 1.3. Consider the asset class forecasts in Exhibit 1.
(a) Which asset class did GMO estimate to have a negative 10-year return over 2002-2011?
(b) Which asset classes substantially outperformed GMO's estimate over that time period?
(c) Which asset classes substantially underperformed GMO's estimate over that time period?
4. Fund Performance.
(a) In which asset class was GMWAX most heavily allocated throughout the majority of 19972011?
(b) Comment on the performance of GMWAX versus its benchmark. (No calculation needed; simply comment on the comparison in the exhibits.)



# 2 Analyzing GMO

This section utilizes data in the file, gmo_data.xlsx.
Examine GMO's performance. Use the risk-free rate to convert the total returns to excess returns



In [158]:
file_path = r"data/gmo_data.xlsx" 
dfs_raw = pd.read_excel(file_path, sheet_name=None)
for key in dfs_raw.keys():
    print(f"{key}: {dfs_raw[key].shape}")

# display(dfs_raw['descriptions'].head())
annual_factor = 12

# ticker_mapping = {tick: name 
#                   for tick, name in zip(dfs_raw['descriptions'].iloc[:, 0], 
#                                                 dfs_raw['descriptions'].iloc[:, 1])}

# ticker_mapping


info: (7, 2)
signals: (335, 4)
risk-free rate: (335, 2)
total returns: (335, 4)


In [159]:
df_risk_free = dfs_raw['risk-free rate'].set_index("date") / annual_factor
df_excess_returns = dfs_raw['total returns'].set_index("date")
df_excess_returns = df_excess_returns.subtract(df_risk_free["TBill 3M"],axis=0)

print(df_excess_returns.shape)
df_excess_returns.head()

(335, 3)


,SPY,GMWAX,GMGEX
date,,,
1996-12-31,-0.0276,-0.0264,-0.0173
1997-01-31,0.0575,0.0104,0.0302
1997-02-28,0.0052,0.0179,0.0084
1997-03-31,-0.0502,-0.0196,-0.0209
1997-04-30,0.0600,-0.0111,-0.0044



## 2.1. Calculate the mean, volatility, and Sharpe ratio for GMWAX. Do this for three samples:

- from inception through 2011
- 2012-present
- inception - present

Has the mean, vol, and Sharpe changed much since the case?


In [160]:
summary_cols = ["mean", "vol", "sharpe", "min", "5% VaR", "max_drawdown"]

masks = {":2011": (df_excess_returns.index.year <= 2011),
         "2012:": (df_excess_returns.index.year >= 2012),
         "full": df_excess_returns.index,
}


main_col = "GMWAX"
df_summary = pd.DataFrame()
for sample_name, sample_mask in masks.items():
    df_temp = cpm.calc_summary_stats(df_excess_returns[[main_col]],
                                      mask=sample_mask, 
                                      annual_factor=annual_factor,
                                      summary_cols=summary_cols
                                      ).T
    df_temp["mask"] = sample_name
    df_summary = pd.concat([df_summary, df_temp])
df_summary = df_summary.set_index("mask")
df_summary

,mean,vol,sharpe,min,5% VaR,max_drawdown
mask,,,,,,
:2011,0.0464,0.1105,0.4201,-0.1492,-0.0440,0.3065
2012:,0.0434,0.0949,0.4573,-0.1150,-0.0409,0.2256
full,0.0450,0.1035,0.4352,-0.1492,-0.0414,0.3065


## 2.2. GMO believes a risk premium is compensation for a security's tendency to lose money at "bad times". For all three samples, analyze extreme scenarios by looking at

- Min return
- 5th percentile (VaR-5th)
- Maximum drawdown ${ }^{1}$
(a) Does GMWAX have high or low tail-risk as seen by these stats?
(b) Does that vary much across the two subsamples?




## 2.3. For all three samples, regress excess returns of GMWAX on excess returns of SPY.
(a) Report the estimated alpha, beta, and r-squared.
(b) Is GMWAX a low-beta strategy? Has that changed since the case?
(c) Does GMWAX provide alpha? Has that changed across the subsamples?


In [161]:
def factor_model(df, y_var, x_vars, intercept=True, lag=0):
    
    if not isinstance(x_vars, list):
        x_vars = [x_vars]
    # Run regression
    formula = f"{y_var} ~ {' + '.join(x_vars)}"
    if not intercept:
        formula = formula + " - 1"
    model = smf.ols(formula=formula, data=df)
    results = model.fit()
    summary = results.summary()

    return model, results

def risk_summary(df, market_col, index_cols=None, annual_scale=12):
    if index_cols is None:
        index_cols = list(df.columns)
        if market_col in index_cols: index_cols.remove(market_col)


    data = []
    for c in index_cols:
        _, results = factor_model(df, c, market_col)
        market_beta = results.params[market_col]

        mu_return = df[c].mean() * annual_scale
        vol_return = df[c].std() * np.sqrt(annual_scale)

        data.append({
            "index": c,
            "market_alpha": results.params["Intercept"],
            "market_beta": market_beta,
            "trenor_ratio": mu_return / market_beta,
            "info_ratio": mu_return / vol_return,
            "R2": results.rsquared,
        })
    return pd.DataFrame(data).set_index("index")


main_col = "GMWAX"
market_col = "SPY"
df_temp = pd.DataFrame()
for sample_name, sample_mask in masks.items():
    df_temp2 = risk_summary(df_excess_returns.loc[sample_mask,[main_col, market_col]], market_col)
    df_temp2["mask"] = sample_name
    df_temp = pd.concat([df_temp, df_temp2])
df_temp = df_temp.set_index("mask")
df_temp

,market_alpha,market_beta,trenor_ratio,info_ratio,R2
mask,,,,,
:2011,0.0023,0.5421,0.0856,0.4201,0.6487
2012:,-0.0028,0.5818,0.0746,0.4573,0.7487
full,0.0000,0.5526,0.0815,0.4352,0.6802


In [162]:
df_excess_returns['GMGEX'].mean() * 12

-0.0014629411533982493


## 2.4. Above, we've evaluated GMO's macro fund, "GMWAX", as studied in the case. Now, consider GMO's equity fund, "GMGEX".
Compute the performance stats of 3.1-3.3 for GMGEX. What are some of the major differences between these two strategies?


In [163]:
main_col = "GMGEX"

# Summary
df_summary = pd.DataFrame()
for sample_name, sample_mask in masks.items():
    df_temp = cpm.calc_summary_stats(df_excess_returns[[main_col]],
                                      mask=sample_mask, 
                                      annual_factor=annual_factor,
                                      summary_cols=summary_cols
                                      ).T
    df_temp["mask"] = sample_name
    df_summary = pd.concat([df_summary, df_temp])
df_summary = df_summary.set_index("mask")
display(df_summary)


# Market Regressions
df_temp = pd.DataFrame()
for sample_name, sample_mask in masks.items():
    df_temp2 = risk_summary(df_excess_returns.loc[sample_mask,[main_col, market_col]], market_col)
    df_temp2["mask"] = sample_name
    df_temp = pd.concat([df_temp, df_temp2])
df_temp = df_temp.set_index("mask")
df_temp

,mean,vol,sharpe,min,5% VaR,max_drawdown
mask,,,,,,
:2011,-0.0038,0.1473,-0.0260,-0.1516,-0.0823,0.5640
2012:,0.0013,0.2356,0.0056,-0.6589,-0.0680,0.7383
full,-0.0015,0.1926,-0.0076,-0.6589,-0.0762,0.7681


,market_alpha,market_beta,trenor_ratio,info_ratio,R2
mask,,,,,
:2011,-0.0026,0.7642,-0.0050,-0.0260,0.7259
2012:,-0.0092,0.8381,0.0016,0.0056,0.2525
full,-0.0054,0.7867,-0.0019,-0.0076,0.3979



# 3 Forecast Regressions

This section utilizes data in the file, gmo_data.xlsx.




## 3.1. Consider the lagged regression, where the regressor, $(X,)$ is a period behind the target, $\left(r^{S P Y}\right)$.

$$
\begin{equation*}
r_{t}^{S P Y}=\alpha^{S P Y, \boldsymbol{X}}+\left(\boldsymbol{\beta}^{S P Y, \boldsymbol{X}}\right)^{\prime} \boldsymbol{X}_{t-1}+\epsilon_{t}^{S P Y, \boldsymbol{X}} \tag{1}
\end{equation*}
$$

Estimate (1) and report the $\mathcal{R}^{2}$, as well as the OLS estimates for $\alpha$ and $\beta$. Do this for...

[^0]- $\boldsymbol{X}$ as a single regressor, the dividend-price ratio.
- $\boldsymbol{X}$ as a single regressor, the earnings-price ratio.
- $\boldsymbol{X}$ as three regressors, the dividend-price ratio, the earnings-price ratio, and the 10 -year yield.

For each, report the r-squared.


In [164]:
def model_print(results):
    return pd.Series({
        "R2": results.rsquared,
        **results.params
    })


df_signals = dfs_raw['signals'].set_index("date")
df_reg = df_excess_returns.merge(df_signals, left_index=True, right_index=True)
df_reg.columns = [c.replace(" ", "_").replace("/","_") for c in df_reg.columns]

# Lag "SPY"
lag = 1
df_reg = df_reg.copy()
df_reg["SPY"] = df_reg["SPY"].shift(lag)
df_reg = df_reg.dropna()

df_reg.head()

,SPY,GMWAX,GMGEX,SPX_DVD_YLD,SPX_P_E,TNote_10YR
date,,,,,,
1997-01-31,-0.0276,0.0104,0.0302,1.8455,20.8856,6.4940
1997-02-28,0.0575,0.0179,0.0084,1.8502,21.0116,6.5520
1997-03-31,0.0052,-0.0196,-0.0209,1.9427,18.4633,6.9030
1997-04-30,-0.0502,-0.0111,-0.0044,1.8430,19.6004,6.7180
1997-05-30,0.0600,0.0530,0.0539,1.7478,19.9884,6.6590


In [165]:
models = []

In [166]:
# Model 1
model_info = {
    "name": "Regression on Dividend Ratio",
    "y_var": "SPY",
    "x_vars": ["SPX_DVD_YLD"],
}

model, results = factor_model(df_reg, model_info["y_var"], model_info["x_vars"])
model_info["model"] = model
model_info["results"] = results

models.append(model_info)
print(model_info["name"])
model_print(results)

Regression on Dividend Ratio


R2             0.0127
Intercept      0.0295
SPX_DVD_YLD   -0.0125
dtype: float64

In [167]:
# Model 2
model_info = {
    "name": "Regression on Earnings Ratio",
    "y_var": "SPY",
    "x_vars": ["SPX_P_E"],
}

model, results = factor_model(df_reg, model_info["y_var"], model_info["x_vars"])
model_info["model"] = model
model_info["results"] = results

models.append(model_info)
print(model_info["name"])
model_print(results)

Regression on Earnings Ratio


R2           0.0165
Intercept   -0.0194
SPX_P_E      0.0013
dtype: float64

In [168]:
# Model 3
model_info = {
    "name": "Regression on 3 Factors",
    "y_var": "SPY",
    "x_vars": ["SPX_DVD_YLD", "SPX_P_E", "TNote_10YR"],
}

model, results = factor_model(df_reg, model_info["y_var"], model_info["x_vars"])
model_info["model"] = model
model_info["results"] = results

models.append(model_info)
print(model_info["name"])
model_print(results)

Regression on 3 Factors


R2             0.0255
Intercept      0.0203
SPX_DVD_YLD   -0.0107
SPX_P_E        0.0009
TNote_10YR    -0.0032
dtype: float64


## 3.2. For each of the three regressions, let's try to utilize the resulting forecast in a trading strategy.

- Build the forecasted SPY returns: $\hat{r}_{t+1}^{\mathrm{SPY}}$. Note that this denotes the forecast made using $\boldsymbol{X}_{t}$ to forecast the $(t+1)$ return.
- Set the scale of the investment in SPY equal to 100 times the forecasted value:

$$
w_{t}=100 \hat{r}_{t+1}^{\mathrm{SPY}}
$$

We are not taking this scaling too seriously. We just want the strategy to go bigger in periods where the forecast is high and to withdraw in periods where the forecast is low, or even negative.

- Calculate the return on this strategy:

$$
r_{t+1}^{\mathrm{X}}=w_{t} r_{t+1}^{\mathrm{SPY}}
$$

You should now have the trading strategy returns, $r^{\mathrm{x}}$ for each of the forecasts. For each strategy, estimate

- mean, volatility, Sharpe,
- max-drawdown
- market alpha
- market beta
- market Information ratio


In [ ]:
# Run models to predict SPY
df_strategies = pd.DataFrame()
for idx in range(len(models)):
    model_info = models[idx]
    df_strategies[f"strategy_{idx}"] = model_info["results"].predict(df_reg[model_info["x_vars"]])
df_strategies.head()

,strategy_0,strategy_1,strategy_2
date,,,
1997-01-31,0.0064,0.0082,-0.0021
1997-02-28,0.0063,0.0084,-0.0022
1997-03-31,0.0051,0.0050,-0.0065
1997-04-30,0.0064,0.0065,-0.0039
1997-05-30,0.0076,0.0070,-0.0023


In [193]:
# Summary
df_strategies["SPY"] = df_reg["SPY"]
df_summary = cpm.calc_summary_stats(df_strategies,
                                    annual_factor=annual_factor,
                                    summary_cols=summary_cols
                                    ).T
    
df_summary = (df_summary
              .merge(risk_summary(df_strategies, "SPY"), 
                     left_index=True, 
                     right_index=True)
)


df_summary[['mean', 'vol', 'sharpe', '5% VaR', 'max_drawdown', 'market_alpha', 'market_beta', 'info_ratio']]

,mean,vol,sharpe,5% VaR,max_drawdown,market_alpha,market_beta,info_ratio
strategy_0,0.0812,0.0174,4.6551,0.0008,0.1092,0.0067,0.0127,4.6551
strategy_1,0.0812,0.0199,4.0902,-0.0007,0.0288,0.0067,0.0165,4.0902
strategy_2,0.0812,0.0247,3.2888,-0.0023,0.1451,0.0066,0.0255,3.2888




## 3.3. GMO believes a risk premium is compensation for a security's tendency to lose money at "bad times". Let's consider risk characteristics.
(a) For both strategies, the market, and GMO, calculate the monthly VaR for $\pi=.05$. Just use the quantile of the historic data for this VaR calculation.

(b) The GMO case mentions that stocks under-performed short-term bonds from 2000-2011. Does the dynamic portfolio above under-perform the risk-free rate over this time?

(c) Based on the regression estimates, in how many periods do we estimate a negative risk premium?

(d) Do you believe the dynamic strategy takes on extra risk??



# 4 Out-of-Sample Forecasting

This section utilizes data in the file, gmo_data.xlsx.
Reconsider the problem above, of estimating (1) for $\boldsymbol{x}$. The reported $\mathcal{R}^{2}$ was the in-sample $\mathcal{R}^{2}$-it examined how well the forecasts fit in the sample from which the parameters were estimated.
In particular, focus on the case of using both dividend-price and earnings-price as signals.
Let's consider the out-of-sample r-squared. To do so, we need to do the following:

- Start at $t=60$.
- Estimate (1) only using data through time $t$.
- Use the estimated parameters of (1), along with $\boldsymbol{x}_{t+1}$ to calculate the out-of-sample forecast for the following period, $t+1$.

$$
\hat{r}_{t+1}^{S P Y}=\hat{\alpha}_{t}^{S P Y, \boldsymbol{x}}+\left(\boldsymbol{\beta}^{S P Y, \boldsymbol{x}}\right)^{\prime} \boldsymbol{x}_{t}
$$

- Calculate the $t+1$ forecast error,

$$
e_{t+1}^{\text {forecast }}=r_{t+1}^{S P Y}-\hat{r}_{t+1}^{S P Y}
$$

- Move to $t=61$, and loop through the rest of the sample.

You now have the time-series of out-of-sample prediction errors, $e^{x}$.
Calculate the time-series of out-of-sample prediction errors $e^{\text {null }}$, which are based on the null forecast:

$$
\begin{aligned}
\bar{r}_{t+1}^{S P Y} & =\frac{1}{t} \sum_{i=1}^{t} r_{i}^{S P Y} \\
e_{t+1}^{\mathrm{null}} & =r_{t+1}^{S P Y}-\bar{r}_{t+1}^{S P Y}
\end{aligned}
$$




## 4.1. Report the out-of-sample $\mathcal{R}^{2}$ :

$$
\mathcal{R}_{O O S}^{2} \equiv 1-\frac{\sum_{i=61}^{T}\left(e_{i}^{\text {forecast }}\right)^{2}}{\sum_{i=61}^{T}\left(e_{i}^{\text {null }}\right)^{2}}
$$

Note that unlike an in-sample r-squared, the out-of-sample r-squared can be anywhere between $(-\infty, 1]$.
Did this forecasting strategy produce a positive OOS r-squared?



## 4.. Re-do problem 3.2 using this OOS forecast.

How much better/worse is the OOS strategy compared to the in-sample version of 3.2?



## 4.. Re-do problem 3.3 using this OOS forecast.

Is the point-in-time version of the strategy riskier?



# 5 Extensions

This section is not graded, and you do not need to submit your answers. We may discuss some of these extensions.




## 5.1. Classification and Regression Tree (CART)

Re-do Section 3, but use CART in forecasting instead of a lagged regression.

- Consider using RandomForestRegressor from sklearn.ensemble.
- If you want to plot the tree, try using tree from sklearn.




## 5.2. Expand on the CART analysis by calculating the OOS stats, as in Section 4.



## 5.3. Re-do Section 3, but use a Neural Network in forecasting instead of a lagged regression.

- Consider using MLPRegressor from sklearn.neural_network.




## 5.3. Re-do Section 3, but use a Neural Network in forecasting instead of a lagged regression.

- Consider using MLPRegressor from sklearn.neural_network.




## 5.4. Expand on the CART and Neural Network analysis by calculating the OOS stats, as in Section 4.

[^0]:    ${ }^{1}$ This should be calculated on GMWAX total returns, not excess returns.

